In [2]:
import tensorflow as tf
import numpy as np

In [3]:
import util

In [5]:
#train_images = np.load('./lstm_data/train_images.npy') 
train_states = np.load('./lstm_data/train_states.npy')
train_actions = np.load('./lstm_data/train_actions.npy')

SEQ = 6
BS = 32

In [16]:
train_states[0].shape

(2889, 6, 5)

In [6]:
VALID_ACTIONS = np.asarray([4,5,6,36,37,38])

In [29]:
tf.reset_default_graph()
batch_size = tf.placeholder_with_default(tf.constant(32),())
img_inputs = tf.placeholder_with_default(tf.zeros([BS,SEQ,64,96,3]), (BS, SEQ, 64, 96, 3),name = 'image_input')
state_inputs = tf.placeholder(tf.float32, (None,SEQ,5),name = 'state_input')
action_labels = tf.placeholder_with_default(tf.cast(tf.zeros([BS,SEQ]),tf.int64), (BS,SEQ),name='action_label')
keep_prob = tf.placeholder_with_default(1.0,())

# gray = tf.image.rgb_to_grayscale(img_inputs)
# reshaped_img_inputs = tf.reshape(gray,[-1,64,96,1])

# conv1 = tf.contrib.layers.conv2d(inputs = reshaped_img_inputs, kernel_size = 7, stride = 4, num_outputs = 16, weights_regularizer = tf.nn.l2_loss)
# conv2 = tf.contrib.layers.conv2d(inputs = conv1, kernel_size = 5, stride = 4, num_outputs = 32, weights_regularizer = tf.nn.l2_loss)
# conv3 = tf.contrib.layers.conv2d(inputs = conv2, kernel_size = 3, stride = 2, num_outputs = 4, weights_regularizer = tf.nn.l2_loss)
# cnn_out = tf.contrib.layers.flatten(conv3)

# final_img_inputs = tf.reshape(cnn_out,[BS,SEQ,cnn_out.shape[1]])

# img_lstm = tf.contrib.rnn.LSTMCell(20)
state_lstm = tf.contrib.rnn.LSTMCell(50)

# with tf.variable_scope('dynamic_img'):
#     img_lstm_outputs, _ = tf.nn.dynamic_rnn(img_lstm, final_img_inputs, dtype=tf.float32)
#     img_lstm_outputs = tf.reshape(img_lstm_outputs,[BS,SEQ,-1])
    
with tf.variable_scope('dynamic_state1'):
    state_lstm_outputs,_ = tf.nn.dynamic_rnn(state_lstm, state_inputs, dtype = tf.float32)
    #state_lstm_outputs = tf.reshape(state_lstm_outputs,[batch_size,SEQ,-1])
    print(state_lstm_outputs.shape)
    
state_lstm1 = tf.contrib.rnn.LSTMCell(50)
with tf.variable_scope('dynamic_state3'):
    state_lstm_outputs1,_ = tf.nn.dynamic_rnn(state_lstm1, state_lstm_outputs, dtype = tf.float32)
    #state_lstm_outputs2 = tf.reshape(state_lstm_outputs,[BS,SEQ,-1])
#print(state_lstm_outputs1.shape)
    
# state_lstm2 = tf.contrib.rnn.LSTMCell(6)
# with tf.variable_scope('dynamic_state2'):
#     state_lstm_outputs2,_ = tf.nn.dynamic_rnn(state_lstm2, state_lstm_outputs1, dtype = tf.float32)
#     #state_lstm_outputs2 = tf.reshape(state_lstm_outputs,[BS,SEQ,-1])



    
#final_concat = tf.concat([img_lstm_outputs,state_lstm_outputs],axis = 2)
final_concat = tf.identity(state_lstm_outputs1)
fc1 = tf.contrib.layers.fully_connected(inputs = final_concat,num_outputs = 512,weights_regularizer = tf.nn.l2_loss)
dropout = tf.nn.dropout(fc1,keep_prob = keep_prob)
output = tf.contrib.layers.fully_connected(inputs = dropout,num_outputs = VALID_ACTIONS.shape[0],activation_fn=None,weights_regularizer =tf.nn.l2_loss)
#print(output.shape)
#output =  tf.identity(state_lstm_outputs2)
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = output, labels = action_labels)+1e-5*sum(tf.nn.l2_loss(tf_var)
        for tf_var in tf.trainable_variables()
        if not ("noreg" in tf_var.name or "Bias" in tf_var.name))
action = tf.identity(tf.argmax(output,axis = 2),name = 'action')
accuracy = tf.reduce_mean(tf.cast(tf.equal(action,action_labels),tf.float32))
optimizer = tf.train.AdamOptimizer(0.001, 0.9, 0.999)
trainer = optimizer.minimize(loss)
print("Total number of variables:",np.sum([v.get_shape().num_elements() for v in tf.trainable_variables()]))

(?, 6, 50)
Total number of variables: 60590


Training

In [8]:
#val_images = np.load('./human_lstm_data/test_images.npy')[0] 
val_states = np.load('./lstm_data/test_states.npy')[0]
val_actions = np.load('./lstm_data/test_actions.npy')[0]

image 10 LSTM state 100 LSTM

## HUMAN RESULTS

In [28]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)

sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

ES = 1000
N_BATCHES = 50
for e in range(ES):
    total_acc = []
    total_loss = []
    for set_i in range(train_states.shape[0]):
        #curr_imgs = train_images[set_i]
        curr_states = train_states[set_i]
        curr_actions = train_actions[set_i]
        
        
        example_indeces = np.random.choice(curr_states.shape[0],N_BATCHES*BS,replace=True)
        
        for batch in range(N_BATCHES):
            batch_idx = example_indeces[batch*BS:(batch+1)*BS]
            #I = curr_imgs[batch_idx]
            S = curr_states[batch_idx]
            A = curr_actions[batch_idx]
            #print(A)
            #print(A)
            out,batch_acc,batch_loss,_ = sess.run([action,accuracy,loss,trainer],{state_inputs:S,action_labels:A,keep_prob:1})
            
            #break
            total_acc.append(batch_acc)
            total_loss.append(batch_loss)
        
    
    #print('['+str(e)+']','\t acc:',np.mean(total_acc),'\t loss:',np.mean(total_loss))
    val_acc = []
    val_loss = []
    for batch in range(0,int(val_states.shape[0]/BS)):
        #I = val_images[batch*BS:(batch+1)*BS]
        S = val_states[batch*BS:(batch+1)*BS]
        A = val_actions[batch*BS:(batch+1)*BS]
        acc,v_loss = sess.run([accuracy,loss],{state_inputs:S,action_labels:A,keep_prob:1})
        
        val_acc.append(acc)
        val_loss.append(v_loss)
    
    print('['+str(e)+']',' \t t.acc:',np.mean(total_acc),'\t t.loss:',np.mean(total_loss), '\t v.acc',np.mean(val_acc),'\t v.loss',np.mean(val_loss))
    if (np.mean(val_acc)>0.96):
        util.save('kart_lstm_human.tfg',tf.get_default_graph(),sess)
    #break
    #break
        
        

[0]  	 t.acc: 0.458056 	 t.loss: 1.50745 	 v.acc 0.439373 	 v.loss 1.55719
[1]  	 t.acc: 0.509931 	 t.loss: 1.35411 	 v.acc 0.447331 	 v.loss 1.55585
[2]  	 t.acc: 0.521979 	 t.loss: 1.31128 	 v.acc 0.447566 	 v.loss 1.5495
[3]  	 t.acc: 0.526979 	 t.loss: 1.28575 	 v.acc 0.447917 	 v.loss 1.52383
[4]  	 t.acc: 0.547604 	 t.loss: 1.23912 	 v.acc 0.463074 	 v.loss 1.5009
[5]  	 t.acc: 0.575278 	 t.loss: 1.17901 	 v.acc 0.461201 	 v.loss 1.55327
[6]  	 t.acc: 0.577361 	 t.loss: 1.16105 	 v.acc 0.451486 	 v.loss 1.55002
[7]  	 t.acc: 0.594931 	 t.loss: 1.11434 	 v.acc 0.485487 	 v.loss 1.52396
[8]  	 t.acc: 0.592708 	 t.loss: 1.11672 	 v.acc 0.477177 	 v.loss 1.51122
[9]  	 t.acc: 0.609236 	 t.loss: 1.07796 	 v.acc 0.459328 	 v.loss 1.57208
[10]  	 t.acc: 0.607465 	 t.loss: 1.06545 	 v.acc 0.457573 	 v.loss 1.56809
[11]  	 t.acc: 0.613646 	 t.loss: 1.06104 	 v.acc 0.466643 	 v.loss 1.53659
[12]  	 t.acc: 0.619931 	 t.loss: 1.04891 	 v.acc 0.476709 	 v.loss 1.53761
[13]  	 t.acc: 0.630694 

[108]  	 t.acc: 0.833576 	 t.loss: 0.454457 	 v.acc 0.461376 	 v.loss 3.02606
[109]  	 t.acc: 0.837396 	 t.loss: 0.450013 	 v.acc 0.454412 	 v.loss 3.05456
[110]  	 t.acc: 0.838576 	 t.loss: 0.449106 	 v.acc 0.46243 	 v.loss 3.09542
[111]  	 t.acc: 0.839861 	 t.loss: 0.437511 	 v.acc 0.461084 	 v.loss 3.12156
[112]  	 t.acc: 0.835486 	 t.loss: 0.452954 	 v.acc 0.466994 	 v.loss 3.02357
[113]  	 t.acc: 0.834826 	 t.loss: 0.449158 	 v.acc 0.46518 	 v.loss 3.00061
[114]  	 t.acc: 0.837639 	 t.loss: 0.452608 	 v.acc 0.459328 	 v.loss 3.14054
[115]  	 t.acc: 0.842813 	 t.loss: 0.437453 	 v.acc 0.460908 	 v.loss 3.17894
[116]  	 t.acc: 0.835208 	 t.loss: 0.447097 	 v.acc 0.462488 	 v.loss 3.1924
[117]  	 t.acc: 0.842396 	 t.loss: 0.433814 	 v.acc 0.464712 	 v.loss 3.12098
[118]  	 t.acc: 0.847222 	 t.loss: 0.425958 	 v.acc 0.461903 	 v.loss 3.13274
[119]  	 t.acc: 0.840208 	 t.loss: 0.438958 	 v.acc 0.466409 	 v.loss 3.1397
[120]  	 t.acc: 0.845764 	 t.loss: 0.431329 	 v.acc 0.474017 	 v.los

[214]  	 t.acc: 0.885938 	 t.loss: 0.332405 	 v.acc 0.467463 	 v.loss 4.31252
[215]  	 t.acc: 0.881215 	 t.loss: 0.334946 	 v.acc 0.459621 	 v.loss 4.3301
[216]  	 t.acc: 0.883194 	 t.loss: 0.330912 	 v.acc 0.465356 	 v.loss 4.29335
[217]  	 t.acc: 0.880694 	 t.loss: 0.334571 	 v.acc 0.461318 	 v.loss 4.38052
[218]  	 t.acc: 0.883576 	 t.loss: 0.335863 	 v.acc 0.465766 	 v.loss 4.36343
[219]  	 t.acc: 0.88559 	 t.loss: 0.324673 	 v.acc 0.462956 	 v.loss 4.43954
[220]  	 t.acc: 0.884722 	 t.loss: 0.333053 	 v.acc 0.469862 	 v.loss 4.42452
[221]  	 t.acc: 0.883194 	 t.loss: 0.334522 	 v.acc 0.473549 	 v.loss 4.39618
[222]  	 t.acc: 0.886806 	 t.loss: 0.324959 	 v.acc 0.469335 	 v.loss 4.45857
[223]  	 t.acc: 0.886875 	 t.loss: 0.325164 	 v.acc 0.465414 	 v.loss 4.49643
[224]  	 t.acc: 0.891285 	 t.loss: 0.317504 	 v.acc 0.467989 	 v.loss 4.53523
[225]  	 t.acc: 0.884687 	 t.loss: 0.331313 	 v.acc 0.475246 	 v.loss 4.4311
[226]  	 t.acc: 0.885278 	 t.loss: 0.329965 	 v.acc 0.466351 	 v.lo

[320]  	 t.acc: 0.904097 	 t.loss: 0.280862 	 v.acc 0.457104 	 v.loss 5.3499
[321]  	 t.acc: 0.910278 	 t.loss: 0.271455 	 v.acc 0.461259 	 v.loss 5.31125
[322]  	 t.acc: 0.908195 	 t.loss: 0.273 	 v.acc 0.465063 	 v.loss 5.2895
[323]  	 t.acc: 0.909861 	 t.loss: 0.269706 	 v.acc 0.467697 	 v.loss 5.39503
[324]  	 t.acc: 0.909201 	 t.loss: 0.276079 	 v.acc 0.461669 	 v.loss 5.35515
[325]  	 t.acc: 0.906354 	 t.loss: 0.28319 	 v.acc 0.462313 	 v.loss 5.32802
[326]  	 t.acc: 0.90559 	 t.loss: 0.279855 	 v.acc 0.464595 	 v.loss 5.30232
[327]  	 t.acc: 0.906667 	 t.loss: 0.277852 	 v.acc 0.466643 	 v.loss 5.46605
[328]  	 t.acc: 0.908194 	 t.loss: 0.27793 	 v.acc 0.473783 	 v.loss 5.28051
[329]  	 t.acc: 0.907674 	 t.loss: 0.277827 	 v.acc 0.46758 	 v.loss 5.3999
[330]  	 t.acc: 0.905174 	 t.loss: 0.29075 	 v.acc 0.456636 	 v.loss 5.42883
[331]  	 t.acc: 0.905972 	 t.loss: 0.27619 	 v.acc 0.458041 	 v.loss 5.51677
[332]  	 t.acc: 0.909965 	 t.loss: 0.27004 	 v.acc 0.465356 	 v.loss 5.41356

[426]  	 t.acc: 0.921111 	 t.loss: 0.249611 	 v.acc 0.464654 	 v.loss 6.05242
[427]  	 t.acc: 0.922917 	 t.loss: 0.240813 	 v.acc 0.459738 	 v.loss 6.11662
[428]  	 t.acc: 0.924688 	 t.loss: 0.236738 	 v.acc 0.466117 	 v.loss 6.05573
[429]  	 t.acc: 0.924618 	 t.loss: 0.241422 	 v.acc 0.460616 	 v.loss 6.16635
[430]  	 t.acc: 0.915 	 t.loss: 0.260205 	 v.acc 0.45055 	 v.loss 6.13215
[431]  	 t.acc: 0.916181 	 t.loss: 0.254413 	 v.acc 0.46401 	 v.loss 6.20335
[432]  	 t.acc: 0.921424 	 t.loss: 0.244717 	 v.acc 0.452306 	 v.loss 6.18111
[433]  	 t.acc: 0.921528 	 t.loss: 0.242628 	 v.acc 0.459621 	 v.loss 6.13122
[434]  	 t.acc: 0.921875 	 t.loss: 0.239279 	 v.acc 0.45611 	 v.loss 6.13907
[435]  	 t.acc: 0.92066 	 t.loss: 0.244687 	 v.acc 0.459972 	 v.loss 6.19313
[436]  	 t.acc: 0.925382 	 t.loss: 0.234346 	 v.acc 0.470272 	 v.loss 6.18054
[437]  	 t.acc: 0.923368 	 t.loss: 0.24304 	 v.acc 0.45728 	 v.loss 6.22742
[438]  	 t.acc: 0.923125 	 t.loss: 0.23966 	 v.acc 0.456051 	 v.loss 6.20

[532]  	 t.acc: 0.92934 	 t.loss: 0.234727 	 v.acc 0.462547 	 v.loss 6.72893
[533]  	 t.acc: 0.929479 	 t.loss: 0.224289 	 v.acc 0.454998 	 v.loss 6.75074
[534]  	 t.acc: 0.930694 	 t.loss: 0.221829 	 v.acc 0.456636 	 v.loss 6.72666
[535]  	 t.acc: 0.932465 	 t.loss: 0.217591 	 v.acc 0.458567 	 v.loss 6.75048
[536]  	 t.acc: 0.932396 	 t.loss: 0.223085 	 v.acc 0.45254 	 v.loss 6.78542
[537]  	 t.acc: 0.92941 	 t.loss: 0.225408 	 v.acc 0.455115 	 v.loss 6.62199
[538]  	 t.acc: 0.923819 	 t.loss: 0.248319 	 v.acc 0.450843 	 v.loss 6.83035
[539]  	 t.acc: 0.927882 	 t.loss: 0.227682 	 v.acc 0.453944 	 v.loss 6.76699
[540]  	 t.acc: 0.929062 	 t.loss: 0.230208 	 v.acc 0.452949 	 v.loss 6.7372
[541]  	 t.acc: 0.927465 	 t.loss: 0.228269 	 v.acc 0.454705 	 v.loss 6.71915
[542]  	 t.acc: 0.93441 	 t.loss: 0.217734 	 v.acc 0.454939 	 v.loss 6.69023
[543]  	 t.acc: 0.931076 	 t.loss: 0.220142 	 v.acc 0.454764 	 v.loss 6.75024
[544]  	 t.acc: 0.930139 	 t.loss: 0.222888 	 v.acc 0.458216 	 v.loss

[638]  	 t.acc: 0.936979 	 t.loss: 0.209002 	 v.acc 0.440543 	 v.loss 7.27712
[639]  	 t.acc: 0.93507 	 t.loss: 0.21234 	 v.acc 0.44938 	 v.loss 7.19521
[640]  	 t.acc: 0.935903 	 t.loss: 0.214118 	 v.acc 0.451486 	 v.loss 7.21544
[641]  	 t.acc: 0.931597 	 t.loss: 0.218851 	 v.acc 0.446161 	 v.loss 7.11886
[642]  	 t.acc: 0.936215 	 t.loss: 0.210031 	 v.acc 0.445868 	 v.loss 7.34059
[643]  	 t.acc: 0.935937 	 t.loss: 0.20933 	 v.acc 0.446337 	 v.loss 7.40637
[644]  	 t.acc: 0.935556 	 t.loss: 0.216531 	 v.acc 0.449146 	 v.loss 7.33914
[645]  	 t.acc: 0.937639 	 t.loss: 0.207838 	 v.acc 0.446278 	 v.loss 7.24252
[646]  	 t.acc: 0.938646 	 t.loss: 0.205159 	 v.acc 0.454529 	 v.loss 7.24897
[647]  	 t.acc: 0.936493 	 t.loss: 0.214143 	 v.acc 0.451311 	 v.loss 7.15488
[648]  	 t.acc: 0.938542 	 t.loss: 0.206595 	 v.acc 0.448619 	 v.loss 7.16378
[649]  	 t.acc: 0.93559 	 t.loss: 0.216074 	 v.acc 0.449906 	 v.loss 7.35814
[650]  	 t.acc: 0.937118 	 t.loss: 0.211696 	 v.acc 0.445049 	 v.loss

KeyboardInterrupt: 

In [16]:
val_states.shape

(2871, 6, 5)

In [10]:
graph = util.load('kart_lstm.tfg')

INFO:tensorflow:Restoring parameters from /tmp/tmpky5t79z8/kart_lstm.tfg


In [29]:
util.save('kart_lstm_human.tfg',tf.get_default_graph(),sess)

In [32]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)

sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

ES = 1000
N_BATCHES = 50
max_ = 0.96
for e in range(ES):
    total_acc = []
    total_loss = []
    for set_i in range(train_states.shape[0]):
        #curr_imgs = train_images[set_i]
        curr_states = train_states[set_i]
        curr_actions = train_actions[set_i]
        
        
        example_indeces = np.random.choice(curr_states.shape[0],N_BATCHES*BS,replace=True)
        
        for batch in range(N_BATCHES):
            batch_idx = example_indeces[batch*BS:(batch+1)*BS]
            #I = curr_imgs[batch_idx]
            S = curr_states[batch_idx]
            A = curr_actions[batch_idx]
            #print(A)
            #print(A)
            out,batch_acc,batch_loss,_ = sess.run([action,accuracy,loss,trainer],{state_inputs:S,action_labels:A,keep_prob:0.5})
            
            #break
            total_acc.append(batch_acc)
            total_loss.append(batch_loss)
        
    
    #print('['+str(e)+']','\t acc:',np.mean(total_acc),'\t loss:',np.mean(total_loss))
    val_acc = []
    val_loss = []
    for batch in range(0,int(val_states.shape[0]/BS)):
        #I = val_images[batch*BS:(batch+1)*BS]
        S = val_states[batch*BS:(batch+1)*BS]
        A = val_actions[batch*BS:(batch+1)*BS]
        acc,v_loss = sess.run([accuracy,loss],{state_inputs:S,action_labels:A,keep_prob:1})
        
        val_acc.append(acc)
        val_loss.append(v_loss)
    
    print('['+str(e)+']',' \t t.acc:',np.mean(total_acc),'\t t.loss:',np.mean(total_loss), '\t v.acc',np.mean(val_acc),'\t v.loss',np.mean(val_loss))
    if (np.mean(val_acc)>max_):
        util.save('kart_lstm_%.3f.tfg'%np.mean(val_acc),tf.get_default_graph(),sess)
        max_ = np.mean(val_acc)
    #break
    #break
        
        

[0]  	 t.acc: 0.601432 	 t.loss: 0.964366 	 v.acc 0.672504 	 v.loss 0.698251
[1]  	 t.acc: 0.779714 	 t.loss: 0.521356 	 v.acc 0.882992 	 v.loss 0.326093
[2]  	 t.acc: 0.89737 	 t.loss: 0.284797 	 v.acc 0.926365 	 v.loss 0.219795
[3]  	 t.acc: 0.923359 	 t.loss: 0.21889 	 v.acc 0.937859 	 v.loss 0.192092
[4]  	 t.acc: 0.933854 	 t.loss: 0.192736 	 v.acc 0.945402 	 v.loss 0.176818
[5]  	 t.acc: 0.943125 	 t.loss: 0.174801 	 v.acc 0.948725 	 v.loss 0.164692
[6]  	 t.acc: 0.94638 	 t.loss: 0.166366 	 v.acc 0.951778 	 v.loss 0.163492
[7]  	 t.acc: 0.95138 	 t.loss: 0.152916 	 v.acc 0.950162 	 v.loss 0.161642
[8]  	 t.acc: 0.951563 	 t.loss: 0.15189 	 v.acc 0.955011 	 v.loss 0.14917
[9]  	 t.acc: 0.952578 	 t.loss: 0.145639 	 v.acc 0.953394 	 v.loss 0.153853
[10]  	 t.acc: 0.954818 	 t.loss: 0.142708 	 v.acc 0.956178 	 v.loss 0.150764
[11]  	 t.acc: 0.95599 	 t.loss: 0.132443 	 v.acc 0.958244 	 v.loss 0.14581
[12]  	 t.acc: 0.958385 	 t.loss: 0.130687 	 v.acc 0.958423 	 v.loss 0.141611
[13]

[105]  	 t.acc: 0.982604 	 t.loss: 0.0576824 	 v.acc 0.959411 	 v.loss 0.162675
[106]  	 t.acc: 0.982266 	 t.loss: 0.0580613 	 v.acc 0.961746 	 v.loss 0.166643
[107]  	 t.acc: 0.979688 	 t.loss: 0.0620813 	 v.acc 0.959501 	 v.loss 0.170404
[108]  	 t.acc: 0.980859 	 t.loss: 0.0619665 	 v.acc 0.959501 	 v.loss 0.172885
[109]  	 t.acc: 0.983099 	 t.loss: 0.0568205 	 v.acc 0.961746 	 v.loss 0.169565
[110]  	 t.acc: 0.982891 	 t.loss: 0.0588664 	 v.acc 0.96426 	 v.loss 0.165338
[111]  	 t.acc: 0.981797 	 t.loss: 0.06021 	 v.acc 0.963003 	 v.loss 0.170997
[112]  	 t.acc: 0.982552 	 t.loss: 0.0581952 	 v.acc 0.960399 	 v.loss 0.167869
[113]  	 t.acc: 0.980339 	 t.loss: 0.062537 	 v.acc 0.960309 	 v.loss 0.170396
[114]  	 t.acc: 0.984297 	 t.loss: 0.0531148 	 v.acc 0.961207 	 v.loss 0.167728
[115]  	 t.acc: 0.981719 	 t.loss: 0.0612937 	 v.acc 0.959231 	 v.loss 0.172842
[116]  	 t.acc: 0.983516 	 t.loss: 0.0563547 	 v.acc 0.963542 	 v.loss 0.160687
[117]  	 t.acc: 0.984349 	 t.loss: 0.0553494

[208]  	 t.acc: 0.987891 	 t.loss: 0.0443467 	 v.acc 0.961835 	 v.loss 0.212073
[209]  	 t.acc: 0.986328 	 t.loss: 0.0475694 	 v.acc 0.95968 	 v.loss 0.22706
[210]  	 t.acc: 0.986979 	 t.loss: 0.0480645 	 v.acc 0.957884 	 v.loss 0.230831
[211]  	 t.acc: 0.98862 	 t.loss: 0.0436573 	 v.acc 0.959501 	 v.loss 0.227342
[212]  	 t.acc: 0.987917 	 t.loss: 0.044926 	 v.acc 0.95986 	 v.loss 0.226848
[213]  	 t.acc: 0.986016 	 t.loss: 0.04992 	 v.acc 0.961476 	 v.loss 0.220612
[214]  	 t.acc: 0.986224 	 t.loss: 0.0507902 	 v.acc 0.960758 	 v.loss 0.213322
[215]  	 t.acc: 0.985755 	 t.loss: 0.0507358 	 v.acc 0.95977 	 v.loss 0.224677
[216]  	 t.acc: 0.986589 	 t.loss: 0.0481222 	 v.acc 0.957166 	 v.loss 0.22116
[217]  	 t.acc: 0.986458 	 t.loss: 0.0490262 	 v.acc 0.960129 	 v.loss 0.21347
[218]  	 t.acc: 0.986042 	 t.loss: 0.0499392 	 v.acc 0.962105 	 v.loss 0.209055
[219]  	 t.acc: 0.986641 	 t.loss: 0.0472795 	 v.acc 0.961566 	 v.loss 0.21429
[220]  	 t.acc: 0.986953 	 t.loss: 0.0467105 	 v.ac

[312]  	 t.acc: 0.988203 	 t.loss: 0.0434609 	 v.acc 0.959231 	 v.loss 0.24518
[313]  	 t.acc: 0.988854 	 t.loss: 0.0420895 	 v.acc 0.961027 	 v.loss 0.235116
[314]  	 t.acc: 0.988464 	 t.loss: 0.043229 	 v.acc 0.959501 	 v.loss 0.233469
[315]  	 t.acc: 0.989713 	 t.loss: 0.0410249 	 v.acc 0.956268 	 v.loss 0.25845
[316]  	 t.acc: 0.988724 	 t.loss: 0.0425163 	 v.acc 0.956268 	 v.loss 0.264399
[317]  	 t.acc: 0.988177 	 t.loss: 0.0442487 	 v.acc 0.95968 	 v.loss 0.242505
[318]  	 t.acc: 0.989479 	 t.loss: 0.0413413 	 v.acc 0.956627 	 v.loss 0.255004
[319]  	 t.acc: 0.989557 	 t.loss: 0.0422154 	 v.acc 0.958154 	 v.loss 0.248544
[320]  	 t.acc: 0.988411 	 t.loss: 0.0440866 	 v.acc 0.95977 	 v.loss 0.245819
[321]  	 t.acc: 0.98888 	 t.loss: 0.0432078 	 v.acc 0.958782 	 v.loss 0.247401
[322]  	 t.acc: 0.98849 	 t.loss: 0.0425406 	 v.acc 0.957166 	 v.loss 0.244474
[323]  	 t.acc: 0.98987 	 t.loss: 0.0415111 	 v.acc 0.958333 	 v.loss 0.252143
[324]  	 t.acc: 0.987839 	 t.loss: 0.0458116 	 v

[416]  	 t.acc: 0.989818 	 t.loss: 0.0406848 	 v.acc 0.959501 	 v.loss 0.257956
[417]  	 t.acc: 0.989089 	 t.loss: 0.0417824 	 v.acc 0.959231 	 v.loss 0.236905
[418]  	 t.acc: 0.991068 	 t.loss: 0.0384746 	 v.acc 0.957076 	 v.loss 0.286446
[419]  	 t.acc: 0.991068 	 t.loss: 0.0376077 	 v.acc 0.958423 	 v.loss 0.269586
[420]  	 t.acc: 0.991276 	 t.loss: 0.0368454 	 v.acc 0.956717 	 v.loss 0.269635
[421]  	 t.acc: 0.988906 	 t.loss: 0.0415587 	 v.acc 0.958333 	 v.loss 0.257538
[422]  	 t.acc: 0.990339 	 t.loss: 0.0389458 	 v.acc 0.958333 	 v.loss 0.262059
[423]  	 t.acc: 0.990234 	 t.loss: 0.0399542 	 v.acc 0.958782 	 v.loss 0.272885
[424]  	 t.acc: 0.990391 	 t.loss: 0.0382644 	 v.acc 0.954831 	 v.loss 0.272075
[425]  	 t.acc: 0.990651 	 t.loss: 0.0396359 	 v.acc 0.955909 	 v.loss 0.258653
[426]  	 t.acc: 0.989349 	 t.loss: 0.0403902 	 v.acc 0.957345 	 v.loss 0.28255
[427]  	 t.acc: 0.989089 	 t.loss: 0.0412335 	 v.acc 0.953305 	 v.loss 0.280921
[428]  	 t.acc: 0.989661 	 t.loss: 0.0415

[519]  	 t.acc: 0.990885 	 t.loss: 0.037249 	 v.acc 0.957076 	 v.loss 0.271161
[520]  	 t.acc: 0.992474 	 t.loss: 0.0330264 	 v.acc 0.95537 	 v.loss 0.278377
[521]  	 t.acc: 0.991458 	 t.loss: 0.0359053 	 v.acc 0.957615 	 v.loss 0.286582
[522]  	 t.acc: 0.991146 	 t.loss: 0.0369114 	 v.acc 0.960578 	 v.loss 0.27398
[523]  	 t.acc: 0.991562 	 t.loss: 0.037405 	 v.acc 0.955729 	 v.loss 0.277499
[524]  	 t.acc: 0.989479 	 t.loss: 0.0413323 	 v.acc 0.956807 	 v.loss 0.286936
[525]  	 t.acc: 0.991016 	 t.loss: 0.0375596 	 v.acc 0.959231 	 v.loss 0.265166
[526]  	 t.acc: 0.992162 	 t.loss: 0.0346854 	 v.acc 0.958782 	 v.loss 0.272226
[527]  	 t.acc: 0.990937 	 t.loss: 0.037668 	 v.acc 0.957974 	 v.loss 0.280483
[528]  	 t.acc: 0.990781 	 t.loss: 0.0370473 	 v.acc 0.954023 	 v.loss 0.30037
[529]  	 t.acc: 0.990885 	 t.loss: 0.0373584 	 v.acc 0.958603 	 v.loss 0.271551
[530]  	 t.acc: 0.990755 	 t.loss: 0.0376325 	 v.acc 0.956268 	 v.loss 0.303369
[531]  	 t.acc: 0.990521 	 t.loss: 0.0403905 	

[622]  	 t.acc: 0.992057 	 t.loss: 0.0347455 	 v.acc 0.959141 	 v.loss 0.281478
[623]  	 t.acc: 0.991432 	 t.loss: 0.0361349 	 v.acc 0.957525 	 v.loss 0.279535
[624]  	 t.acc: 0.991198 	 t.loss: 0.0373496 	 v.acc 0.95555 	 v.loss 0.281371
[625]  	 t.acc: 0.991328 	 t.loss: 0.0366 	 v.acc 0.957076 	 v.loss 0.280937
[626]  	 t.acc: 0.991406 	 t.loss: 0.0352456 	 v.acc 0.957705 	 v.loss 0.271875
[627]  	 t.acc: 0.991016 	 t.loss: 0.0359587 	 v.acc 0.957346 	 v.loss 0.285188
[628]  	 t.acc: 0.992213 	 t.loss: 0.0349853 	 v.acc 0.955998 	 v.loss 0.287476
[629]  	 t.acc: 0.992109 	 t.loss: 0.0352426 	 v.acc 0.95959 	 v.loss 0.275992
[630]  	 t.acc: 0.990937 	 t.loss: 0.0373141 	 v.acc 0.95546 	 v.loss 0.286646
[631]  	 t.acc: 0.9925 	 t.loss: 0.0356462 	 v.acc 0.957884 	 v.loss 0.292531
[632]  	 t.acc: 0.991927 	 t.loss: 0.0352322 	 v.acc 0.958962 	 v.loss 0.279866
[633]  	 t.acc: 0.993073 	 t.loss: 0.0330368 	 v.acc 0.958872 	 v.loss 0.281515
[634]  	 t.acc: 0.992057 	 t.loss: 0.0354045 	 v

[725]  	 t.acc: 0.991823 	 t.loss: 0.0367669 	 v.acc 0.956717 	 v.loss 0.295339
[726]  	 t.acc: 0.992474 	 t.loss: 0.0336616 	 v.acc 0.957615 	 v.loss 0.2792
[727]  	 t.acc: 0.992474 	 t.loss: 0.0340595 	 v.acc 0.957256 	 v.loss 0.300585
[728]  	 t.acc: 0.992839 	 t.loss: 0.0331679 	 v.acc 0.960219 	 v.loss 0.274847
[729]  	 t.acc: 0.993021 	 t.loss: 0.0325244 	 v.acc 0.961297 	 v.loss 0.266577
[730]  	 t.acc: 0.990391 	 t.loss: 0.0377317 	 v.acc 0.959411 	 v.loss 0.266085
[731]  	 t.acc: 0.992813 	 t.loss: 0.0331117 	 v.acc 0.958423 	 v.loss 0.276202
[732]  	 t.acc: 0.992422 	 t.loss: 0.0343705 	 v.acc 0.956897 	 v.loss 0.291351
[733]  	 t.acc: 0.991641 	 t.loss: 0.0365776 	 v.acc 0.956807 	 v.loss 0.281969
[734]  	 t.acc: 0.992526 	 t.loss: 0.0340033 	 v.acc 0.956088 	 v.loss 0.296197
[735]  	 t.acc: 0.991667 	 t.loss: 0.0361684 	 v.acc 0.957256 	 v.loss 0.287907
[736]  	 t.acc: 0.99151 	 t.loss: 0.0356529 	 v.acc 0.961566 	 v.loss 0.271843
[737]  	 t.acc: 0.992266 	 t.loss: 0.034922

[828]  	 t.acc: 0.991354 	 t.loss: 0.0355873 	 v.acc 0.958243 	 v.loss 0.269596
[829]  	 t.acc: 0.993229 	 t.loss: 0.0319898 	 v.acc 0.955909 	 v.loss 0.288863
[830]  	 t.acc: 0.993203 	 t.loss: 0.0319589 	 v.acc 0.957705 	 v.loss 0.277179
[831]  	 t.acc: 0.992813 	 t.loss: 0.0325032 	 v.acc 0.955909 	 v.loss 0.281208
[832]  	 t.acc: 0.992292 	 t.loss: 0.0360417 	 v.acc 0.956807 	 v.loss 0.280074
[833]  	 t.acc: 0.992318 	 t.loss: 0.0348026 	 v.acc 0.956897 	 v.loss 0.280226
[834]  	 t.acc: 0.993151 	 t.loss: 0.0314253 	 v.acc 0.956358 	 v.loss 0.290226
[835]  	 t.acc: 0.992448 	 t.loss: 0.0334208 	 v.acc 0.955729 	 v.loss 0.27443
[836]  	 t.acc: 0.993307 	 t.loss: 0.0314695 	 v.acc 0.956986 	 v.loss 0.296602
[837]  	 t.acc: 0.992344 	 t.loss: 0.0338763 	 v.acc 0.958782 	 v.loss 0.284889
[838]  	 t.acc: 0.992917 	 t.loss: 0.0329369 	 v.acc 0.957435 	 v.loss 0.287913
[839]  	 t.acc: 0.993568 	 t.loss: 0.0312032 	 v.acc 0.958872 	 v.loss 0.282083
[840]  	 t.acc: 0.99224 	 t.loss: 0.03413

[931]  	 t.acc: 0.992422 	 t.loss: 0.0338261 	 v.acc 0.956986 	 v.loss 0.293796
[932]  	 t.acc: 0.993021 	 t.loss: 0.0318156 	 v.acc 0.955729 	 v.loss 0.288318
[933]  	 t.acc: 0.993672 	 t.loss: 0.0309227 	 v.acc 0.955909 	 v.loss 0.304859
[934]  	 t.acc: 0.993177 	 t.loss: 0.0317179 	 v.acc 0.956986 	 v.loss 0.298302
[935]  	 t.acc: 0.992969 	 t.loss: 0.0319957 	 v.acc 0.95546 	 v.loss 0.305896
[936]  	 t.acc: 0.99362 	 t.loss: 0.0310648 	 v.acc 0.956537 	 v.loss 0.327332
[937]  	 t.acc: 0.993255 	 t.loss: 0.0326292 	 v.acc 0.957884 	 v.loss 0.299661
[938]  	 t.acc: 0.993073 	 t.loss: 0.0317483 	 v.acc 0.958872 	 v.loss 0.278456
[939]  	 t.acc: 0.99362 	 t.loss: 0.0302439 	 v.acc 0.958782 	 v.loss 0.279901
[940]  	 t.acc: 0.992708 	 t.loss: 0.032831 	 v.acc 0.959231 	 v.loss 0.288657
[941]  	 t.acc: 0.99349 	 t.loss: 0.0310659 	 v.acc 0.958243 	 v.loss 0.283809
[942]  	 t.acc: 0.994219 	 t.loss: 0.0296143 	 v.acc 0.952496 	 v.loss 0.328496
[943]  	 t.acc: 0.993073 	 t.loss: 0.0314261 

In [86]:
val_images.shape

(1, 1873, 6, 64, 96, 3)

In [94]:
val_images[1:3].shape

(2, 6, 64, 96, 3)

In [95]:
val_actions.shape

(1, 1873, 6, 5)

In [98]:
val_actions[0][0]

array([-1.75424599,  0.44162969, -1.78788821, -0.05483892, -1.86871687])